In [1]:
#!pip install selenium 

In [2]:
#!pip install beautifulsoup4 --quiet

In [3]:
#!pip3 install webdriver-manager

In [32]:
location_tosearch = "India"
job_category= "Data Science"

In [33]:
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import ActionChains



In [34]:
# Open browser
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# Opening linkedin/jobs website
driver.get("https://in.linkedin.com/jobs")
## waiting load the webpage , can change depending on internet speed.
time.sleep(2)

#Maximizing browser window to avoid hidden elements
driver.set_window_size(1024, 600)
driver.maximize_window()

In [35]:

job_search_element = "keywords"


In [36]:
from selenium.webdriver.common.by import By

In [37]:
#find the job-search button!
button_xpath = "//*[@id=\"main-content\"]/section[1]/div/section/div[2]/button[2]"
button_element = driver.find_element(By.XPATH, button_xpath)
#type job_category in search field
driver.find_element(By.NAME, job_search_element).send_keys(job_category)
#location
delete_current_location = driver.find_element(By.XPATH,"//*[@id=\"JOBS\"]/section[2]/button/icon").click()
location_element = driver.find_element(By.XPATH, "//*[@id=\"JOBS\"]/section[2]/input")
location_element.send_keys(location_tosearch)





In [38]:

#click search button!
webdriver.ActionChains(driver).move_to_element(button_element ).click(button_element ).perform()

In [39]:
#can be lowered depdending on computer,internet speed
# time to load the page
time.sleep(3)



In [40]:
#saving the url for job_category search 
job_search_url = driver.current_url
    
SCROLL_PAUSE_TIME = 5 #5 seconds
time.sleep(SCROLL_PAUSE_TIME)

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

for x in range(0, 10):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    driver.find_element(By.XPATH, "//body").send_keys(Keys.ARROW_UP)
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    print('current Y: ' + str(new_height))
    if new_height == last_height:
        break
    last_height = new_height

current Y: 4211


In [41]:
#empty data-structures to save data
visited_url = set () #check in constant time
job_details_dict = {}

def save_data(company_url, company_name, job_location, job_role,description,employee):
    job_details_list = [company_name, job_location, job_role,description,company_url,employee]
    job_details_dict[company_url] = job_details_list

In [42]:
job_post_urls = []
page_source = BeautifulSoup(driver.page_source)
job_post = page_source.find_all("a", class_ ="base-card__full-link")
for url in job_post:
    url = url.get("href")
    job_post_urls.append(url)
    
    
    
    
    

In [43]:
#visit the company page with company url to get description
def get_discrition(url):
    counter = 0
    while counter <5:
    
        
        print("Driver -> Company Page")
        
        company_page = driver.get(url)
        time.sleep(4)
        current_url = driver.current_url
        
        if current_url == url:
            description = "n/a"
            employee = ""
            print("getting Data)")
            #desc_xpath = "//*[@id=\"main-content\"]/section[*]/div/section[*]/div/p"
            #desc_element = driver.find_element(By.XPATH, desc_xpath)
            #desc_element = driver.find_element(By.CLASS_NAME, "about-us__description",)                break-words whitespace-pre-wrap
            try:
                desc_element = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "break-words whitespace-pre-wrap",)))
                #desc_element = driver.find_element(By.CLASS_NAME, "break-words whitespace-pre-wrap",)
                description = (desc_element.get_attribute('innerHTML')).strip()
                employee_expath = "//*[@id=\"main-content\"]/section[1]/section/div/div[2]/div[2]/ul/li/div/a"
                employee_element = driver.find_element(By.XPATH, employee_expath)
                employee = (employee_element.get_attribute('innerHTML')).strip()
                
                
            except TimeoutException as timeout:
                
                try:
                
                    #desc_xpath = "//*[@id=\"main-content\"]/section[*]/div/section[*]/div/p"
                    #desc_element = driver.find_element(By.XPATH, desc_xpath)
                    desc_element = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "about-us__description",)))
                    #desc_element = driver.find_element(By.CLASS_NAME, "about-us__description",)
                    description = (desc_element.get_attribute('innerHTML')).strip()

                except TimeoutException as timeout:
                    return (description, employee)
            #time.sleep(2)
            #print("closing window")
            #driver.close()
            #print("switching to original_tab")

            #driver.switch_to.window(original_window)

            return (description,employee)
        else:
            #driver.close()
            time.sleep(10)
            #driver.switch_to.window(driver.window_handles[1])
            driver.switch_to.new_window('tab')
            driver.switch_to.window(driver.window_handles[0])
            driver.close()
            driver.switch_to.window(driver.window_handles[0])

            time.sleep(2)
            
            counter+=1
            continue
            
    

In [44]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoSuchWindowException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException


In [ ]:

for url in job_post_urls:

    if url in visited_url:
        print("already visited")
        continue
    #driver.switch_to.new_window('tab')
    driver.get(url)
    time.sleep(2)

            #Getting jobrole!
    job_role_xpath = "//*[@id=\"main-content\"]/section[1]/div/section[2]/div/div[1]/div/h1"
    job_role_element = driver.find_element(By.XPATH, job_role_xpath)
    job_role = (job_role_element.get_attribute('innerHTML')).strip()
    print(job_role)    

                #getting job location!
    job_location_xpath = "//*[@id=\"main-content\"]/section[1]/div/section[2]/div/div[1]/div/h4/div[1]/span[2]"
    job_location_element = driver.find_element(By.XPATH, job_location_xpath)
    job_location = (job_location_element.get_attribute('innerHTML')).strip()
    print(job_location)

                #getting company name and url 
    company_name_xpath = "//*[@id=\"main-content\"]/section[1]/div/section[2]/div/div[1]/div/h4/div[1]/span[1]/a"
    company_name_element = driver.find_element(By.XPATH, company_name_xpath)
    company_name = (company_name_element.get_attribute('innerHTML')).strip()
    company_url = company_name_element.get_attribute('href')
    print(company_name)
    time.sleep(2)
    #print("closing window")
    #driver.close()
    print("Switching to original tab")
    #driver.switch_to.window(original_window)
    visited_url.add(url)
    #time.sleep(2)




    description,employee = get_discrition(company_url)
    save_data(company_url, company_name, job_location, job_role, description,employee) 

Data Scientist
India
HRPG Intern
Switching to original tab
Driver -> Company Page
getting Data)
Data Science Intern
India
Corizo
Switching to original tab
Driver -> Company Page
getting Data)
Data Scientist
Bengaluru, Karnataka, India
Rolls-Royce
Switching to original tab
Driver -> Company Page
getting Data)
Data Scientist
Chennai, Tamil Nadu, India
Rekha Khurana
Switching to original tab
Driver -> Company Page
getting Data)
Deep Learning Engineer
Bangalore Urban, Karnataka, India
Samsung Electro-Mechanics
Switching to original tab
Driver -> Company Page
getting Data)
Machine Learning Intern
India
Corizo
Switching to original tab
Driver -> Company Page
getting Data)
Artificial Intelligence Intern
India
Teachnook
Switching to original tab
Driver -> Company Page
getting Data)
Data Science Trainee
New Delhi, Delhi, India
MedTourEasy
Switching to original tab
Driver -> Company Page
getting Data)
Machine Learning Engineer
Bengaluru, Karnataka, India
ReFier
Switching to original tab
Driver -

In [ ]:

len(job_details_dict)